In [ ]:
2+2

In [1]:
from data_retriever import DataRetriever
from pre_processing import PreProcessing
from scalers.min_max import MinMax
from regressors.lstm_regressor import LSTMRegressor
import pandas as pd
from model_evaluator import ModelEvaluator

In [3]:
#Obtaining stock market data
a = DataRetriever()
a.get_yahoo_stock_data()   
b = a.get_stock_ds()
x_cols_ = [x for x in b.columns.difference(['Date']) if x[-5:] == 'Close']

for i in x_cols_:
    b = a.get_stock_ds()
    print('Learning to predict stock : ' + i)
    y_cols = i
    x_cols = b.columns.difference(['Date'] + list(y_cols))

    pp = PreProcessing(b, MinMax)
    train, test = pp.pre_process()

    reg = LSTMRegressor(train, x_cols, y_cols, 7, len(x_cols), 40)
    reg.fit_generator()

    reg_pred = reg.predict_generator(test)
    reg_pred = pp._scalers[y_cols].inverse_transform(reg_pred)
    pred = pd.DataFrame(reg_pred)
    pred.columns = ['Prediction']

    begin_idx  = len(b) - len(pred)
    finish_idx = len(b) - 1

    pred.index = list(range(begin_idx, finish_idx + 1))

    b = b.join(pred)

    me = ModelEvaluator('LSTM', b, 'Prediction', y_cols, 'Date')
    me.plot_results(False, True)

NameError: name 'DataRetriever' is not defined